In [29]:
import scanpy as sc
import pandas as pd
import random
import numpy as np
import matplotlib as plt
import gc 
import ctypes
import scvi
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import gc 
import ctypes

In [30]:
adata = sc.read_h5ad('../data/luca_query_reannotated.h5ad')

In [31]:
adata.obs

,sample,uicc_stage,ever_smoker,age,donor_id,origin,dataset,ann_fine,cell_type_predicted,doublet_status,...,IFN_TAMs,Reg_TAMs,Inflam_TAMs,LA_TAMs,Angio_TAMs,RTM_TAMs,Prolif_TAMs,Subtype,Projection_CellType,ident
001C_AAACCTGTCAACACCA-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage alveolar,Macrophage,singlet,...,0.353933,0.571664,0.298638,0.541069,0.427279,0.565822,0.401695,Reg_TAMs,RTM_TAMs,local
001C_AAACGGGAGGCTCATT-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage,Macrophage,singlet,...,0.477315,0.451494,0.370148,0.492239,0.537751,0.511095,0.348617,Angio_TAMs,LA_TAMs,local
001C_AAACGGGGTATAATGG-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage alveolar,Macrophage,singlet,...,0.387389,0.497444,0.282365,0.403790,0.443018,0.548354,0.328568,RTM_TAMs,RTM_TAMs,local
001C_AAACGGGTCACAAACC-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage,Macrophage,singlet,...,0.333177,0.592321,0.262999,0.594631,0.444907,0.450925,0.406192,LA_TAMs,LA_TAMs,local
001C_AAAGATGAGTGCTGCC-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage alveolar,Macrophage,singlet,...,0.358754,0.648105,0.282604,0.482257,0.408499,0.454457,0.291259,Reg_TAMs,RTM_TAMs,local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAAGCTGTCTA-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,0.551482,0.640531,0.489101,0.623875,0.525708,0.514441,0.527691,Reg_TAMs,IFN_TAMs,local
TTTGTCAAGGATATAC-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,0.492014,0.699073,0.539889,0.612688,0.608443,0.551273,0.460248,Reg_TAMs,Reg_TAMs,local
TTTGTCAAGTACGCGA-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,0.470449,0.588994,0.611164,0.427515,0.471975,0.431026,0.455491,Inflam_TAMs,Inflam_TAMs,local
TTTGTCACATCTATGG-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,0.515349,0.589936,0.633147,0.577088,0.550079,0.487074,0.425062,Inflam_TAMs,LA_TAMs,local


In [32]:
adata.obs['Projection_CellType'].value_counts()


Projection_CellType
RTM_TAMs       94074
LA_TAMs        32428
Reg_TAMs       17933
IFN_TAMs       12215
Inflam_TAMs     6484
Angio_TAMs      5741
Int.Node.3      4221
Prolif_TAMs     3771
Int.Node.4       922
Int.Node.5       150
Name: count, dtype: int64

In [ ]:
#In case the adata of pseudo-bulk data needs to be saved.
import pandas as pd
import scipy.sparse

def savePBdata2(adata, filename):
    # Convert counts to dense matrix if it's sparse
    if scipy.sparse.issparse(adata.X):
        counts = adata.X.toarray()
    else:
        counts = adata.X

    # Convert to DataFrame
    counts_df = pd.DataFrame(counts, index=adata.obs_names, columns=adata.var_names)

    # (Optional) Convert to integer
    counts_df = counts_df.astype(int, errors='ignore')

    # Save AnnData object
    adata.write(filename)
    
    # Save counts as CSV for inspection (optional)
    counts_df.to_csv(filename.replace(".h5ad", "_counts.csv"))

    print(f"Saved AnnData to {filename} and count matrix to CSV.")


In [12]:
#Pseudo-bulking with random down sampling once. min_cells=40, max_cells=50
ps_adata_macs = make_pseudobulk3(adata, sample_colname = 'sample', cell_type_colname = 'Projection_CellType', metacols= ['assay', 'donor_id', 'disease', 'tissue', 'study', 'sex', 'age', 'uicc_stage', 'tumor_stage'])

/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use 

In [13]:
ps_adata_macs.obs.shape

(613, 12)

In [14]:
ps_adata_macs.obs

,Projection_CellType,assay,donor_id,disease,tissue,study,sex,age,uicc_stage,tumor_stage,n_cells,log2_n_cells
Adams_Kaminski_2020_001C_RTM_TAMs,RTM_TAMs,10x 3' v2,Adams_Kaminski_2020_001C,normal,lung,Adams_Kaminski_2020,male,22.0,non-cancer,non-cancer,50,5.643856
Adams_Kaminski_2020_001C_LA_TAMs,LA_TAMs,10x 3' v2,Adams_Kaminski_2020_001C,normal,lung,Adams_Kaminski_2020,male,22.0,non-cancer,non-cancer,50,5.643856
Adams_Kaminski_2020_001C_Int.Node.3,Int.Node.3,10x 3' v2,Adams_Kaminski_2020_001C,normal,lung,Adams_Kaminski_2020,male,22.0,non-cancer,non-cancer,45,5.491853
Adams_Kaminski_2020_002C_RTM_TAMs,RTM_TAMs,10x 3' v2,Adams_Kaminski_2020_002C,normal,lung,Adams_Kaminski_2020,female,25.0,non-cancer,non-cancer,50,5.643856
Adams_Kaminski_2020_002C_LA_TAMs,LA_TAMs,10x 3' v2,Adams_Kaminski_2020_002C,normal,lung,Adams_Kaminski_2020,female,25.0,non-cancer,non-cancer,50,5.643856
...,...,...,...,...,...,...,...,...,...,...,...,...
Leader_Merad_2021_414_RTM_TAMs,RTM_TAMs,10x 3' v2,Leader_Merad_2021_729,lung adenocarcinoma,lung,Leader_Merad_2021,female,64.0,II,early,50,5.643856
Leader_Merad_2021_414_Angio_TAMs,Angio_TAMs,10x 3' v2,Leader_Merad_2021_729,lung adenocarcinoma,lung,Leader_Merad_2021,female,64.0,II,early,50,5.643856
Leader_Merad_2021_414_IFN_TAMs,IFN_TAMs,10x 3' v2,Leader_Merad_2021_729,lung adenocarcinoma,lung,Leader_Merad_2021,female,64.0,II,early,50,5.643856
Leader_Merad_2021_414_Reg_TAMs,Reg_TAMs,10x 3' v2,Leader_Merad_2021_729,lung adenocarcinoma,lung,Leader_Merad_2021,female,64.0,II,early,50,5.643856


In [ ]:
savePBdata2(ps_adata_macs, filename="../data/ps_adata_macs.h5ad")


Saved AnnData to ../data/ps_adata_macs.h5ad and count matrix to CSV.


In [33]:
with open('../data/Homo_sapiens.GRCh38.104.gtf') as f:
    gtf = list(f)

#prep the gtf file
gtf = [x for x in gtf if not x.startswith('#')]
gtf = [x for x in gtf if 'gene_id "' in x and 'gene_name "' in x]
gtf = list(map(lambda x: (x.split('gene_id "')[1].split('"')[0], x.split('gene_name "')[1].split('"')[0]), gtf))

## Downsample - Pseudo-bulk - DEG

In [66]:
from joblib import Parallel, delayed
from scipy.sparse import issparse
import multiprocessing
multiprocessing.set_start_method("spawn", force=True)

def bulk_one_group(adata, group_key, idx, metacols, cell_type_colname, min_cells, max_cells, rng):
    s, ct = group_key
    cell_indices = np.array(idx)
    n_cells_total = len(cell_indices)

    if n_cells_total < min_cells:
        return None

    selected = rng.choice(cell_indices, size=min(max_cells, n_cells_total), replace=False)
    sub = adata[selected]

    summed_X = sub.X.sum(axis=0)
    if issparse(sub.X):
        summed_X = summed_X.A1

    adata_rep = sc.AnnData(X=summed_X.reshape(1, -1), var=sub.var[[]])
    adata_rep = adata_rep.copy()
    adata_rep.obs_names = [f"{s}_{ct}"]
    adata_rep.obs[cell_type_colname] = ct

    for col in metacols:
        adata_rep.obs[col] = sub.obs[col].iloc[0]

    adata_rep.obs["n_cells"] = len(selected)
    return adata_rep


def make_pseudobulk3_parallel(adata, sample_colname, cell_type_colname, metacols, min_cells=40, max_cells=50, random_state=0, n_jobs=4):
    rng = np.random.default_rng(random_state)
    groups = adata.obs.groupby([sample_colname, cell_type_colname])

    results = Parallel(n_jobs=n_jobs, backend="threading")(
        delayed(bulk_one_group)(
            adata, (s, ct), idx, metacols, cell_type_colname, min_cells, max_cells,
            np.random.default_rng(random_state + i)
        )
        for i, ((s, ct), idx) in enumerate(groups.indices.items())
    )

    ps_list = [res for res in results if res is not None]
    ps_adata = sc.concat(ps_list)
    ps_adata = ps_adata.copy()
    ps_adata.obs["n_cells"] = ps_adata.obs["n_cells"].astype(int)
    ps_adata.obs["log2_n_cells"] = np.log2(ps_adata.obs["n_cells"] + 1)

    return ps_adata


In [65]:
from collections import defaultdict
from joblib import Parallel, delayed
import pandas as pd
import scanpy as sc
import itertools
import multiprocessing
multiprocessing.set_start_method("spawn", force=True)


def run_deseq_for_celltype(ps_adata, c, cell_type_colname, metacols, gtf, condition_col=['Contrast'], latent_factors=['assay']):
    sub_c = ps_adata[ps_adata.obs[cell_type_colname] == c ]
    sub_c = sub_c.copy()
    sub_c.obs[condition_col] = c
    sub_ct = ps_adata[ps_adata.obs[cell_type_colname] != c ]
    sub_ct = sub_ct.copy()
    sub_ct.obs[condition_col] = 'others'
    pb = sc.concat([sub_c, sub_ct])

    counts = pd.DataFrame(pb.X.toarray() if hasattr(pb.X, 'toarray') else pb.X, columns=pb.var_names)
    counts += 1  # Avoid 0s
    
    # Create a design factors:
    if "log2_n_cells" not in latent_factors:
        latent_factors.append("log2_n_cells")
    
    if condition_col not in latent_factors:
        design_factors_local = latent_factors + condition_col
    else:
        design_factors_local = latent_factors[:]
    
    print("design_factors_local:", f"~ {' + '.join(design_factors_local)}")

    dds = DeseqDataSet(counts=counts, 
                       metadata=pb.obs, 
                       design=f"~ {' + '.join(design_factors_local)}", 
                       quiet=True,
                       low_memory=True)
    sc.pp.filter_genes(dds, min_cells=10)
    dds.deseq2()
    stat_res = DeseqStats(dds, contrast=('Contrast', c, 'others'))
    stat_res.summary()

    de = stat_res.results_df
    de['gene_symbols'] = de.index
    de['gene_name'] = de['gene_symbols'].map(dict(gtf)) if gtf else de['gene_symbols']
    de = de[(de['padj'] < 0.05) & (de['log2FoldChange'] > 1.0)]
    de['SubType'] = c
    de['inContrastTo'] = 'others'
    return de

def repeat_deseq_pseudobulk_analysis_parallel(adata, ctyps, sample_colname, cell_type_colname, metacols, N=10, min_cells=40, max_cells=50, min_repeats=5, gtf=None, n_jobs=4):
    gene_hits = defaultdict(int)
    final_results = []

    for i in range(N):
        print(f"=== Iteration {i+1}/{N} ===")
        ps_adata = make_pseudobulk3_parallel(
            adata, 
            sample_colname=sample_colname,
            cell_type_colname=cell_type_colname,
            metacols=metacols,
            min_cells=min_cells,
            max_cells=max_cells,
            random_state=i,
            n_jobs=n_jobs
        )

        results = Parallel(n_jobs=n_jobs, backend="threading")(
            delayed(run_deseq_for_celltype)(ps_adata, c, cell_type_colname, metacols, gtf) 
            for c in ctyps
        )

        for de in results:
            if de is not None and not de.empty:
                for g, c in zip(de['gene_name'], de['SubType']):
                    gene_hits[(g, c)] += 1
                final_results.append(de)

    all_deg_df = pd.concat(final_results)
    stable_genes = [(gene, subtype) for (gene, subtype), count in gene_hits.items() if count >= min_repeats]
    stable_df = all_deg_df[all_deg_df.apply(lambda row: (row['gene_name'], row['SubType']) in stable_genes, axis=1)]

    stable_df.to_csv('../results/macs_PB-DEGs_repeated.csv', index=True)
    return stable_df, gene_hits



In [67]:
ctyps = ['RTM_TAMs', 'LA_TAMs', 'Reg_TAMs', 'IFN_TAMs', 'Angio_TAMs', 'Inflam_TAMs', 'Prolif_TAMs']

stable_degs, gene_hits_dict = repeat_deseq_pseudobulk_analysis_parallel(
    adata=adata,
    ctyps=ctyps,
    sample_colname='sample',
    cell_type_colname='Projection_CellType',
    metacols=['assay', 'donor_id', 'disease', 'tissue', 'study', 'sex', 'age', 'uicc_stage', 'tumor_stage'],
    N=10,  # number of iterations
    min_cells=40,
    max_cells=50,
    min_repeats=9,
    gtf=gtf,  # or None if not available
    n_jobs=60
)


=== Iteration 1/10 ===
design_factors_local: ~ assay + log2_n_cells + Contrast
design_factors_local: ~ assay + log2_n_cells + Contrast
design_factors_local: ~ assay + log2_n_cells + Contrast
design_factors_local: ~ assay + log2_n_cells + Contrast
design_factors_local: ~ assay + log2_n_cells + Contrast
design_factors_local: ~ assay + log2_n_cells + Contrast
design_factors_local: ~ assay + log2_n_cells + Contrast


/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/parallel.py:1359: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/parallel.py:1359: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/parallel.py:1359: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/parallel.py:1359: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/biolab/miniconda3/lib/python3.13/site-packages

Log2 fold change & Wald test p-value: Contrast IFN_TAMs vs others
                  baseMean  log2FoldChange     lfcSE      stat        pvalue  \
ENSG00000121410   8.970604       -0.187295  0.107411 -1.743729  8.120633e-02   
ENSG00000268895   2.108948        0.054376  0.134189  0.405220  6.853159e-01   
ENSG00000175899  42.831185       -0.264706  0.192513 -1.375004  1.691301e-01   
ENSG00000245105   1.306306        0.064160  0.167012  0.384167  7.008547e-01   
ENSG00000166535   1.301070       -0.095156  0.177346 -0.536555  5.915751e-01   
...                    ...             ...       ...       ...           ...   
ENSG00000070476   4.355007       -0.313778  0.155801 -2.013968  4.401286e-02   
ENSG00000203995   1.264108       -0.118468  0.179123 -0.661380  5.083685e-01   
ENSG00000162378   8.329801       -0.527963  0.106791 -4.943892  7.657797e-07   
ENSG00000159840  37.984148        0.064987  0.088978  0.730372  4.651630e-01   
ENSG00000074755   7.975996       -0.376578  0.137589 -

... done in 4.97 seconds.

Running Wald tests...
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/parallel.py:1359: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/parallel.py:1359: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
... done in 39.44 seconds.



Log2 fold change & Wald test p-value: Contrast Inflam_TAMs vs others
                  baseMean  log2FoldChange     lfcSE      stat    pvalue  \
ENSG00000121410   8.970604       -0.021512  0.136495 -0.157605  0.874768   
ENSG00000268895   2.108948        0.018624  0.178668  0.104237  0.916982   
ENSG00000175899  42.831185       -0.311712  0.252638 -1.233828  0.217267   
ENSG00000245105   1.306306       -0.237799  0.243042 -0.978428  0.327863   
ENSG00000166535   1.301070        0.011775  0.229878  0.051222  0.959149   
...                    ...             ...       ...       ...       ...   
ENSG00000070476   4.355007       -0.491534  0.209669 -2.344340  0.019061   
ENSG00000203995   1.264108       -0.092575  0.237525 -0.389747  0.696724   
ENSG00000162378   8.329801       -0.522310  0.143042 -3.651441  0.000261   
ENSG00000159840  37.984148        0.023613  0.116769  0.202224  0.839742   
ENSG00000074755   7.975996       -0.396323  0.181343 -2.185493  0.028853   

                  

...

Log2 fold change & Wald test p-value: Contrast IFN_TAMs vs others
                  baseMean  log2FoldChange     lfcSE      stat    pvalue  \
ENSG00000121410   8.734916       -0.196357  0.106629 -1.841496  0.065549   
ENSG00000268895   2.068758        0.026185  0.136405  0.191965  0.847770   
ENSG00000175899  42.492882       -0.258773  0.191987 -1.347865  0.177702   
ENSG00000245105   1.307645        0.129229  0.166843  0.774555  0.438602   
ENSG00000166535   1.300358       -0.081052  0.177277 -0.457208  0.647522   
...                    ...             ...       ...       ...       ...   
ENSG00000070476   4.386391       -0.385151  0.158488 -2.430161  0.015092   
ENSG00000203995   1.255479       -0.111872  0.180316 -0.620423  0.534979   
ENSG00000162378   8.354758       -0.431382  0.100274 -4.302027  0.000017   
ENSG00000159840  37.918460       -0.025879  0.089205 -0.290107  0.771734   
ENSG00000074755   7.861301       -0.332571  0.137547 -2.417862  0.015612   

                     

/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/parallel.py:1359: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
Fitting dispersions...
... done in 0.03 seconds.

Fitting MAP dispersions...
... done in 0.03 seconds.

Fitting LFCs...
... done in 0.10 seconds.

Running Wald tests...
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/parallel.py:1359: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/parallel.py:1359: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/parallel.py:1359: UserWarning: Loky-backed parallel loops ca

Log2 fold change & Wald test p-value: Contrast LA_TAMs vs others
                  baseMean  log2FoldChange     lfcSE      stat        pvalue  \
ENSG00000121410   8.734916        0.330887  0.075483  4.383582  1.167434e-05   
ENSG00000268895   2.068758       -0.100463  0.102940 -0.975938  3.290950e-01   
ENSG00000175899  42.492882        0.913210  0.139376  6.552117  5.672694e-11   
ENSG00000245105   1.307645       -0.068710  0.127253 -0.539949  5.892321e-01   
ENSG00000166535   1.300358       -0.002738  0.124336 -0.022019  9.824326e-01   
...                    ...             ...       ...       ...           ...   
ENSG00000070476   4.386391       -0.063601  0.113227 -0.561715  5.743105e-01   
ENSG00000203995   1.255479        0.000922  0.126514  0.007291  9.941823e-01   
ENSG00000162378   8.354758       -0.023506  0.071061 -0.330780  7.408106e-01   
ENSG00000159840  37.918460        0.072582  0.066519  1.091140  2.752113e-01   
ENSG00000074755   7.861301        0.112003  0.099081  1

... done in 28.74 seconds.

... done in 28.71 seconds.



Log2 fold change & Wald test p-value: Contrast RTM_TAMs vs others
                  baseMean  log2FoldChange     lfcSE       stat        pvalue  \
ENSG00000121410   8.734916       -0.066055  0.074602  -0.885436  3.759217e-01   
ENSG00000268895   2.068758        0.146427  0.097301   1.504898  1.323504e-01   
ENSG00000175899  42.492882       -2.179706  0.131861 -16.530291  2.220687e-61   
ENSG00000245105   1.307645       -0.070411  0.125854  -0.559469  5.758415e-01   
ENSG00000166535   1.300358       -0.062810  0.126014  -0.498436  6.181771e-01   
...                    ...             ...       ...        ...           ...   
ENSG00000070476   4.386391        0.279493  0.108336   2.579875  9.883619e-03   
ENSG00000203995   1.255479       -0.060931  0.126415  -0.481995  6.298095e-01   
ENSG00000162378   8.354758        0.645598  0.062250  10.371120  3.355682e-25   
ENSG00000159840  37.918460        0.502138  0.063405   7.919586  2.383031e-15   
ENSG00000074755   7.861301        0.090066 

In [39]:
stable_degs['SubType'].value_counts()

SubType
RTM_TAMs       1179
Angio_TAMs      346
Inflam_TAMs     216
Reg_TAMs        193
IFN_TAMs        172
LA_TAMs         100
Prolif_TAMs      12
Name: count, dtype: int64

In [40]:
stable_degs[stable_degs['SubType'] == 'Prolif_TAMs']


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_symbols,gene_name,SubType,inContrastTo
ENSG00000145287,17.068954,1.945502,0.409175,4.754693,1.987485e-06,1.244777e-04,ENSG00000145287,PLAC8,Prolif_TAMs,others
ENSG00000164265,14.052979,1.465126,0.503282,2.911142,3.601108e-03,2.015456e-02,ENSG00000164265,SCGB3A2,Prolif_TAMs,others
ENSG00000122852,9.593422,1.042028,0.428019,2.434536,1.491089e-02,4.821466e-02,ENSG00000122852,SFTPA1,Prolif_TAMs,others
ENSG00000168878,13.160073,1.223473,0.415954,2.941366,3.267678e-03,1.897282e-02,ENSG00000168878,SFTPB,Prolif_TAMs,others
ENSG00000117632,40.219820,2.039778,0.277446,7.351990,1.952781e-13,7.949770e-10,ENSG00000117632,STMN1,Prolif_TAMs,others
ENSG00000197253,5.849828,1.149523,0.392147,2.931357,3.374845e-03,1.942508e-02,ENSG00000197253,TPSB2,Prolif_TAMs,others
ENSG00000145287,18.240100,1.840117,0.414434,4.440072,8.992868e-06,2.908059e-04,ENSG00000145287,PLAC8,Prolif_TAMs,others
ENSG00000164265,14.133490,1.436652,0.506138,2.838456,4.533231e-03,2.109072e-02,ENSG00000164265,SCGB3A2,Prolif_TAMs,others
ENSG00000122852,9.689505,1.034920,0.425396,2.432840,1.498093e-02,4.478995e-02,ENSG00000122852,SFTPA1,Prolif_TAMs,others
ENSG00000168878,13.178998,1.207898,0.417070,2.896149,3.777735e-03,1.877566e-02,ENSG00000168878,SFTPB,Prolif_TAMs,others
